In [247]:
# import pandas as pd
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer, WordNetLemmatizer

# # Download NLTK resources (stopwords and WordNet)
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# # Initialize stemmer and lemmatizer
# stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

# # Function to preprocess text
# def preprocess_text(text):
#     # Tokenize
#     words = nltk.word_tokenize(text.lower())
#     # Remove stopwords
#     words = [word for word in words if word not in stopwords.words('english')]
#     # Stemming
#     #stemmed_words = [stemmer.stem(word) for word in words]
#     #Lemmatization
#     lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
#     return ' '.join( lemmatized_words)

# # Apply preprocessing and add new column



In [248]:
import os
import pyaudio
#import sounddevice as sd
import wave
import numpy as np
import time
import scipy.io.wavfile as wav

In [249]:
def record_audio(file_path, duration=4):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 48000
    RECORD_SECONDS = duration
    WAVE_OUTPUT_FILENAME = file_path
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    print("Recording...")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("Finished recording.")
    stream.stop_stream()
    stream.close()
    audio.terminate()
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [250]:
record_audio("recording/clip_1.wav")


Recording...
Finished recording.


In [251]:
import speech_recognition as sr

# Initialize the recognizer
recognizer = sr.Recognizer()

# Function to convert speech to text
def speech_to_text(audio_file_path):
    with sr.AudioFile(audio_file_path) as source:
        audio_data = recognizer.record(source)  # Record the audio file

    try:
        # Use Google Web Speech API to recognize speech
        text = recognizer.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        print("Google Web Speech API could not understand audio")
    except sr.RequestError as e:
        print(f"Could not request results from Google Web Speech API; {e}")

In [252]:
audio_file_path = "recording/clip_1.wav"
command_to_classify  = speech_to_text(audio_file_path)
print("Text result:",command_to_classify )

Text result: will it rain this specific


In [253]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Load the saved pipeline
pipeline = joblib.load('joblibexp/ClassSvm.joblib')

# Create a DataFrame with a single row containing the command to classify
command_to_classify = input()

data = {'Commands': [command_to_classify]}
single_command_df = pd.DataFrame(data)
#single_command_df['Preprocessed_Commands'] = single_command_df['Commands'].apply(preprocess_text)

# Predict the class
predicted_class = pipeline.predict(single_command_df['Commands'])

print("Predicted class:", predicted_class[0])
print (command_to_classify)

Predicted class: Update
please perform bike update


In [254]:
import pandas as pd
import joblib

# Load the saved model for the predicted class
class_label_to_predict = predicted_class[0]
model_path = f'joblibexp/SubClassmodel_{class_label_to_predict}.joblib'
model = joblib.load(model_path)



# Create a DataFrame with the preprocessed command
data = {'Commands': [command_to_classify]}
single_command_df = pd.DataFrame(data)

# Special handling for "Miscellaneous" and "Not needed" classes
if class_label_to_predict == 'Miscellaneous':
    predicted_subclass = ['Temporary']  # Assign 'Temporary' directly
elif class_label_to_predict == 'Not needed':
    predicted_subclass = ['NC']  # Assign 'NC' directly
else:
    # Predict the subclass for the given command and class
    predicted_subclass = model.predict(single_command_df['Commands'])

print("Predicted subclass for class", class_label_to_predict, ":", predicted_subclass[0])
print (command_to_classify)


Predicted subclass for class Update : Perform
please perform bike update


In [255]:
def sunamdha(command):
    pipeline = joblib.load('joblibexp/ClassSvm.joblib')

    # Create a DataFrame with a single row containing the command to classify
    command_to_classify = command

    data = {'Commands': [command_to_classify]}
    single_command_df = pd.DataFrame(data)
    #single_command_df['Preprocessed_Commands'] = single_command_df['Commands'].apply(preprocess_text)

    # Predict the class
    predicted_class = pipeline.predict(single_command_df['Commands'])

    print("Predicted class:", predicted_class[0])
    return (predicted_class[0])

In [256]:
def sunamdhasubcat(command):
    # Load the saved model for the predicted class
    class_label_to_predict = sunamdha(command)
    model_path = f'joblibexp/SubClassmodel_{class_label_to_predict}.joblib'
    model = joblib.load(model_path)

    # Create a DataFrame with the preprocessed command
    data = {'Commands': [command]}
    single_command_df = pd.DataFrame(data)

    # Special handling for "Miscellaneous" and "Not needed" classes
    if class_label_to_predict == 'Miscellaneous':
        predicted_subclass = ['Temporary']  # Assign 'Temporary' directly
    elif class_label_to_predict == 'Not needed':
        predicted_subclass = ['NC']  # Assign 'NC' directly
    else:
        # Predict the subclass for the given command and class
        predicted_subclass = model.predict(single_command_df['Commands'])

    print("Predicted subclass for class", class_label_to_predict, ":", predicted_subclass[0])
    return([class_label_to_predict, predicted_subclass[0]])

In [257]:
commands = [
"Please check for update",
"Please show me current update",
"please perform bike update",
"please kindly perform bike update",
"will I reach hyderabad with current fuel",
"Check for update",
"Show me current update",
"perform bike update",
"kindly perform bike update",
"Kindly will I reach hyderabad with current fuel",
"Is current fuel enough to reach Hyderabad",
"Are my tyres pressure balanced",
"Show me the fastest route to kolkata",
"Will it rain tomorrow in bangaluru",
"What is the weather status in delhi",
"Play my playlist form spotify",
"Go to goa",
"Please go to goa",
"Take me to goa",
"What is my current battery level",
"Check my current fuel level",
"Do I need to change the tyre",
"Check tyre pressure and balance",
"Check if tyre pressure is balanced",
"what is current temperature",
"turn off the left indicator of the bike",
"turn on the right indicator of the bike",
"Show me the fastest route from X to Y",
"Can you recommend a charging station nearby",
"Will it rain heavily in Delhi today",
"What is the temperature outisde right now",
"Did I receive messages on Instagram",
"Are there any missed calls or app alert",
"Play my favourite songs form Spotify",
"Can you play some background music",
"Terminate pending bike software update",
"Cancel bike system update process",
"Begin bike software system upate",
"Please begin bike software system upate",
"Please check for available system updates for bike",
"Please verify pending bike updates"
]
resultssum=[]
for command in commands:    
    print(command)
    resultssum.append(sunamdhasubcat(command))
resultssum

Please check for update
Predicted class: Update
Predicted subclass for class Update : Check
Please show me current update
Predicted class: Update
Predicted subclass for class Update : Check
please perform bike update
Predicted class: Update
Predicted subclass for class Update : Perform
please kindly perform bike update
Predicted class: Update
Predicted subclass for class Update : Perform
will I reach hyderabad with current fuel
Predicted class: Edge
Predicted subclass for class Edge : Battery/Fuel
Check for update
Predicted class: Update
Predicted subclass for class Update : Check
Show me current update
Predicted class: Update
Predicted subclass for class Update : Check
perform bike update
Predicted class: Update
Predicted subclass for class Update : Perform
kindly perform bike update
Predicted class: Update
Predicted subclass for class Update : Perform
Kindly will I reach hyderabad with current fuel
Predicted class: Edge
Predicted subclass for class Edge : Battery/Fuel
Is current fuel

[['Update', 'Check'],
 ['Update', 'Check'],
 ['Update', 'Perform'],
 ['Update', 'Perform'],
 ['Edge', 'Battery/Fuel'],
 ['Update', 'Check'],
 ['Update', 'Check'],
 ['Update', 'Perform'],
 ['Update', 'Perform'],
 ['Edge', 'Battery/Fuel'],
 ['Edge', 'Battery/Fuel'],
 ['Edge', 'Tires/Tyres'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Weather'],
 ['Cloud', 'Weather'],
 ['Cloud', 'Songs/media'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Traffic/Maps'],
 ['Edge', 'Battery/Fuel'],
 ['Edge', 'Battery/Fuel'],
 ['Edge', 'Tires/Tyres'],
 ['Edge', 'Tires/Tyres'],
 ['Edge', 'Tires/Tyres'],
 ['Cloud', 'Weather'],
 ['Edge', 'Basic'],
 ['Edge', 'Basic'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Weather'],
 ['Cloud', 'Weather'],
 ['Cloud', 'News/Notification'],
 ['Cloud', 'News/Notification'],
 ['Cloud', 'Songs/media'],
 ['Cloud', 'Songs/media'],
 ['Update', 'Cancel'],
 ['Update', 'Cancel'],
 ['Update', 'Perform'],
 ['Update', 'Perform'],
 ['Update', '

In [258]:
resultssum

[['Update', 'Check'],
 ['Update', 'Check'],
 ['Update', 'Perform'],
 ['Update', 'Perform'],
 ['Edge', 'Battery/Fuel'],
 ['Update', 'Check'],
 ['Update', 'Check'],
 ['Update', 'Perform'],
 ['Update', 'Perform'],
 ['Edge', 'Battery/Fuel'],
 ['Edge', 'Battery/Fuel'],
 ['Edge', 'Tires/Tyres'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Weather'],
 ['Cloud', 'Weather'],
 ['Cloud', 'Songs/media'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Traffic/Maps'],
 ['Edge', 'Battery/Fuel'],
 ['Edge', 'Battery/Fuel'],
 ['Edge', 'Tires/Tyres'],
 ['Edge', 'Tires/Tyres'],
 ['Edge', 'Tires/Tyres'],
 ['Cloud', 'Weather'],
 ['Edge', 'Basic'],
 ['Edge', 'Basic'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Traffic/Maps'],
 ['Cloud', 'Weather'],
 ['Cloud', 'Weather'],
 ['Cloud', 'News/Notification'],
 ['Cloud', 'News/Notification'],
 ['Cloud', 'Songs/media'],
 ['Cloud', 'Songs/media'],
 ['Update', 'Cancel'],
 ['Update', 'Cancel'],
 ['Update', 'Perform'],
 ['Update', 'Perform'],
 ['Update', '

In [259]:
df = pd.DataFrame()

# Add Command column
df['Command'] = commands

# Add Subclass Category column
df['Subclass'] = [_ for _, subcat in resultssum]
df['Subclass Category'] = [subcat for _, subcat in resultssum]

# Display DataFrame
print(df)

                                              Command Subclass  \
0                             Please check for update   Update   
1                       Please show me current update   Update   
2                          please perform bike update   Update   
3                   please kindly perform bike update   Update   
4            will I reach hyderabad with current fuel     Edge   
5                                    Check for update   Update   
6                              Show me current update   Update   
7                                 perform bike update   Update   
8                          kindly perform bike update   Update   
9     Kindly will I reach hyderabad with current fuel     Edge   
10          Is current fuel enough to reach Hyderabad     Edge   
11                     Are my tyres pressure balanced     Edge   
12               Show me the fastest route to kolkata    Cloud   
13                 Will it rain tomorrow in bangaluru    Cloud   
14        

In [260]:
df

,Command,Subclass,Subclass Category
0,Please check for update,Update,Check
1,Please show me current update,Update,Check
2,please perform bike update,Update,Perform
3,please kindly perform bike update,Update,Perform
4,will I reach hyderabad with current fuel,Edge,Battery/Fuel
5,Check for update,Update,Check
6,Show me current update,Update,Check
7,perform bike update,Update,Perform
8,kindly perform bike update,Update,Perform
9,Kindly will I reach hyderabad with current fuel,Edge,Battery/Fuel


In [261]:
df.to_csv("SVM_OUTPUT.csv")